In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [2]:
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32) #dropout

W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))
L1 = tf.nn.conv2d(X, W1, strides=[1, 1, 1, 1], padding='SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

W3 = tf.Variable(tf.random_normal([7 * 7 * 64, 256], stddev=0.01))
L3 = tf.reshape(L2, [-1, 7 * 7 * 64])
L3 = tf.matmul(L3, W3)
L3 = tf.nn.relu(L3)

L3 = tf.reshape(L3,[100, 1, 256])

In [3]:
hidden_size = 10
batch_size = 100
cell = tf.contrib.rnn.GRUCell(num_units=hidden_size)
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(
    cell, L3, initial_state=initial_state, dtype=tf.float32)
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y,logits=outputs))
train = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)

total_batch_train = int(mnist.train.num_examples / batch_size)
total_batch_test = int(mnist.test.num_examples / batch_size)
output_list = []
batch_y = []
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(1):
        total_cost = 0
        for i in range(total_batch_train):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        # 이미지 데이터를 CNN 모델을 위한 자료형태인 [28 28 1] 의 형태로 재구성합니다.
            batch_xs = batch_xs.reshape(-1, 28, 28, 1)
            _, cost_val,asd = sess.run([train, loss,outputs],
                               feed_dict={X: batch_xs,
                                          Y: batch_ys,
                                          keep_prob: 0.7})
    for j in range(total_batch_test):
        batch_xs, batch_ys = mnist.test.next_batch(batch_size)
        # 이미지 데이터를 CNN 모델을 위한 자료형태인 [28 28 1] 의 형태로 재구성합니다.
        batch_xs = batch_xs.reshape(-1, 28, 28, 1)
        outout = sess.run(outputs,
                               feed_dict={X: batch_xs,
                                          Y: batch_ys,
                                          keep_prob: 1.0})
        output_list.append(outout)
        batch_y.append(batch_ys)
        

In [4]:
convert_prediction = np.reshape(output_list[0],[100,10])
for i in range(1,100):
     convert_prediction = np.vstack((convert_prediction,np.reshape(output_list[i],[100,10])))

In [5]:
convert_prediction

array([[-0.99999952, -1.        , -0.99999976, ...,  0.99999982,
        -0.99999988, -1.        ],
       [-0.99999994, -0.99697554,  1.        , ..., -1.        ,
        -1.        , -1.        ],
       [-0.99983382,  0.99998736, -0.99978787, ..., -0.99988323,
        -0.99998266, -0.99998337],
       ..., 
       [-0.9999997 , -0.99999994, -0.99998999, ..., -0.99989849,
        -0.99999851, -0.9999783 ],
       [-0.99999833, -0.99999994, -0.99999988, ..., -0.99999994,
        -0.98441952, -0.99999982],
       [-1.        , -0.99999988, -0.99999982, ..., -0.99999976,
        -1.        , -1.        ]], dtype=float32)

In [6]:
convert_batch_y = np.reshape(np.array(batch_y),[10000,10])

In [7]:
convert_batch_y[42]

array([ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.])

In [8]:
convert_batch_y

array([[ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [9]:
convert_prediction

array([[-0.99999952, -1.        , -0.99999976, ...,  0.99999982,
        -0.99999988, -1.        ],
       [-0.99999994, -0.99697554,  1.        , ..., -1.        ,
        -1.        , -1.        ],
       [-0.99983382,  0.99998736, -0.99978787, ..., -0.99988323,
        -0.99998266, -0.99998337],
       ..., 
       [-0.9999997 , -0.99999994, -0.99998999, ..., -0.99989849,
        -0.99999851, -0.9999783 ],
       [-0.99999833, -0.99999994, -0.99999988, ..., -0.99999994,
        -0.98441952, -0.99999982],
       [-1.        , -0.99999988, -0.99999982, ..., -0.99999976,
        -1.        , -1.        ]], dtype=float32)

In [10]:
prediction = np.argmax(convert_prediction, axis=1)
real = np.argmax(convert_batch_y, axis=1)

In [11]:
np.count_nonzero(prediction == real)/10000

0.954